# Training a Model with Hugging Face -> https://huggingface.co/docs/transformers/en/training

In [1]:
from datasets import load_dataset

/Users/geoffreysmalling/development/hugging-face/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# load the yelp dataset of yelp reviews https://huggingface.co/datasets/yelp_review_full


In [2]:
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

The dataset is loaded into ~/.cache/huggingface

Now you need to pre-process the data via padding and truncation and tokenize

Lets load the tokenizer first

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


/Users/geoffreysmalling/development/hugging-face/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


lets build a method to tokenize

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

Let loop through the dataset via MAP and tokeninze

In [5]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map: 100%|█████████████████████████████████████████████████████| 50000/50000 [00:07<00:00, 6973.28 examples/s]


You can simplify the dataset to reduce training time


In [6]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [7]:
print(small_train_dataset[0])

{'label': 4, 'text': "I stalk this truck.  I've been to industrial parks where I pretend to be a tech worker standing in line, strip mall parking lots, and of course the farmer's market.  The bowls are so so absolutely divine.  The owner is super friendly and he makes each bowl by hand with an incredible amount of pride.  You gotta eat here guys!!!", 'input_ids': [101, 146, 27438, 1142, 4202, 119, 146, 112, 1396, 1151, 1106, 3924, 8412, 1187, 146, 9981, 1106, 1129, 170, 13395, 7589, 2288, 1107, 1413, 117, 6322, 8796, 5030, 7424, 117, 1105, 1104, 1736, 1103, 9230, 112, 188, 2319, 119, 1109, 20400, 1132, 1177, 1177, 7284, 10455, 119, 1109, 3172, 1110, 7688, 4931, 1105, 1119, 2228, 1296, 7329, 1118, 1289, 1114, 1126, 10965, 2971, 1104, 8188, 119, 1192, 13224, 3940, 1303, 3713, 106, 106, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [8]:
from transformers import AutoModelForSequenceClassification

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Adjust Model with parameters  TrainingArguments and HyperParameters

In [10]:
from transformers import TrainingArguments
# overide the output training directory
training_args = TrainingArguments(output_dir="test_trainer")

## Trining does not automatically evaluate, you need to pass a function for training..  Evaluate and Accuracy Model

In [11]:
import numpy as np
import evaluate

In [12]:
metric = evaluate.load("accuracy")

Call compute on metric to calculate the accuracy of your predictions. Before passing your predictions to compute, you need to convert the logits to predictions (remember all 🤗 Transformers models return logits):

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

# Create a Trainer model to train the Model

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()


TrainOutput(global_step=375, training_loss=1.1803165690104167, metrics={'train_runtime': 341.3152, 'train_samples_per_second': 8.79, 'train_steps_per_second': 1.099, 'total_flos': 789354427392000.0, 'train_loss': 1.1803165690104167, 'epoch': 3.0})

In [50]:
trainer.push_to_hub("cedralpass/test_trainer")


CommitInfo(commit_url='https://huggingface.co/cedralpass/test_trainer/commit/c9f6d3bb392b598a2aeedbbc1ca8ce788231c62b', commit_message='cedralpass/test_trainer', commit_description='', oid='c9f6d3bb392b598a2aeedbbc1ca8ce788231c62b', pr_url=None, pr_revision=None, pr_num=None)

In [51]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
tokenizer.push_to_hub("cedralpass/test_trainer")

CommitInfo(commit_url='https://huggingface.co/cedralpass/test_trainer/commit/8d6e5f3c5f9bca33e860f0e379dbe218d64d659f', commit_message='Upload tokenizer', commit_description='', oid='8d6e5f3c5f9bca33e860f0e379dbe218d64d659f', pr_url=None, pr_revision=None, pr_num=None)

In [53]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cedralpass/test_trainer")

encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")
print(encoded_input)

{'input_ids': [101, 2091, 1136, 1143, 13002, 1107, 1103, 5707, 1104, 16678, 1116, 117, 1111, 1152, 1132, 11515, 1105, 3613, 1106, 4470, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [57]:
from transformers import pipeline
classifier = pipeline(model="cedralpass/test_trainer")
classifier(
    "I have a problem with my iphone that needs to be resolved asap!!"
)


[{'label': 'LABEL_3', 'score': 0.3528541624546051}]

In [61]:
#unmasker = pipeline('fill-mask', model='cedralpass/test_trainer')
unmasker = pipeline('fill-mask', model='google-bert/bert-base-cased')
unmasker("The [MASK] was spicey.")

Some weights of the model checkpoint at google-bert/bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.24949607253074646,
  'token': 1586,
  'token_str': 'air',
  'sequence': 'The air was spicey.'},
 {'score': 0.1420145481824875,
  'token': 4077,
  'token_str': 'wine',
  'sequence': 'The wine was spicey.'},
 {'score': 0.10141658037900925,
  'token': 5080,
  'token_str': 'taste',
  'sequence': 'The taste was spicey.'},
 {'score': 0.05345313251018524,
  'token': 4773,
  'token_str': 'smell',
  'sequence': 'The smell was spicey.'},
 {'score': 0.03227892145514488,
  'token': 5974,
  'token_str': 'scent',
  'sequence': 'The scent was spicey.'}]

## Todo..  Not sure the pre-training makes sense and how to pick models / save models.  
